# potentially useful code snippets for potential later use

In [2]:
# we put the incoming data in here:
np_eo = dynarray.DynamicArray((None, len(channel_names)))

# init/open the window in which we visualize data
w=guis.AcquireData(sampling_freq, channel_names)

# before we start, pull everything from the buffer (empty it)
data_inlet.pull_chunk()
while data_inlet.samples_available(): data_inlet.pull_chunk() 
    
# then start acquiring data as long as button 'stop' not pressed:
w.RUNLOOP=True
while w.RUNLOOP:


    if not data_inlet.samples_available():
        w.update(None)
    else:
        
        chunk_data, chunk_times = data_inlet.pull_chunk(timeout=0.0) # grab from LSL

        np_eo.extend(chunk_data) # add to our list
        w.update(chunk_data) # update the GUI window        

NameError: name 'dynarray' is not defined